# Installing and starting Oscar

First, you need to install the (compiler for) `julia`. For that, follow the installation 
instructions on the OSCAR website (www.oscar-computeralgebra.de) and make sure you 
install the latest julia version. 

Usually, you would then proceed to follow the installation instructions for 
OSCAR there, but for this course we will need a modified version of OSCAR which comes with some 
extra patches that have not (yet) made their way in the official release. 
To install and use that version, open a julia REPL (Terminal, notebook, ...) and type

In [ ]:
using Pkg
] add "https://github.com/HechtiDerLachs/Oscar.jl#CIMPA_school" #This will take some time 

Then you should be able to start OSCAR with the command 

In [2]:
using Oscar

┌ Info: Precompiling Oscar [f1435218-dba5-11e9-1e4d-f1a5fab5fc13]
└ @ Base loading.jl:1423


 -----    -----    -----      -      -----   
|     |  |     |  |     |    | |    |     |  
|     |  |        |         |   |   |     |  
|     |   -----   |        |     |  |-----   
|     |        |  |        |-----|  |   |    
|     |  |     |  |     |  |     |  |    |   
 -----    -----    -----   -     -  -     -  

...combining (and extending) ANTIC, GAP, Polymake and Singular
Version 0.10.0 ... 
 ... which comes with absolutely no warranty whatsoever
Type: '?Oscar' for more information
(c) 2019-2022 by The Oscar Development Team


### Installing jupyter-notebook

What you see here is a so-called `jupyter` notebook. Those notebooks do not run out of 
the box with julia. On Ubuntu/Debian, you need to install the package `jupyter-notebook`. 
In addition, you need an extension of `julia` so that it can connect to `jupyter`. 
According to the OSCAR website, you are supposed to run

In [ ]:
using Pkg
Pkg.add("IJulia")

### Updating OSCAR during the course

It might be necessary during the course to update OSCAR. To do so, type

In [ ]:
] up

### Switching back to the official OSCAR version after the course

After the course is finished and you wish to continue using OSCAR, you can switch to the 
official release version as follows

In [ ]:
] rm Oscar
using Pkg
Pkg.add("Oscar")

# FIrst steps in OSCAR

## Polynomial rings, etc.

Some standard rings and fields are already there:

In [3]:
QQ

Rational Field

In [4]:
ZZ

Integer Ring

In [5]:
GF(53)

Galois field with characteristic 53

Some others need to be manually created:

In [3]:
R, (x,y,z) = QQ["x", "y", "z"]

(Multivariate Polynomial Ring in x, y, z over Rational Field, fmpq_mpoly[x, y, z])

Note that the above command returns a tuple `(R, v)` consisting of the polynomial ring `R` and another tuple `v = (x,y,z)` containing its variables.

We can now investigate both `R` and the variables:

In [7]:
R

Multivariate Polynomial Ring in x, y, z over Rational Field

In [8]:
x

x

This does not tell us much. But of course, there is more 
going on in the background. For instance, everything in 
julia has a type and so does our variable `x`. We will 
come back to that later.

In [9]:
typeof(x)

fmpq_mpoly

We can also access the variables via the `getindex`-function 
for the ring `R`:

In [10]:
x == R[1]

true

Any ring and its elements in Oscar have a parent-element relationship. We can check which ring an element belongs to:

In [11]:
parent(x)

Multivariate Polynomial Ring in x, y, z over Rational Field

In [12]:
parent(x) == R

true

Let us create another ring over a different field. 
This time, we use another constructor:

In [13]:
S, w = PolynomialRing(GF(101), "w" => 1:5)

(Multivariate Polynomial Ring in w_{1}, w_{2}, w_{3}, w_{4}, w_{5} over Galois field with characteristic 101, gfp_mpoly[w_{1}, w_{2}, w_{3}, w_{4}, w_{5}])

In [14]:
w[3]

w_{3}

In [15]:
parent(w[2]) == R

false

In [16]:
parent(w[2]) == S

true

## Matrices, ideals, and modules

### Matrices over polynomial rings

Given a ring `R`, we can set up matrices over it:

In [17]:
M = R[x y z; z x y^2]

[x   y     z]
[z   x   y^2]

Note that the separation of a rows entries is done by whitespaces. 
Unfortunately, that means that you have to be careful when using
blanks in your expressions:

In [18]:
M = R[x y z; z x y^2] #TODO: Modify and play around to create a mess

[x   y     z]
[z   x   y^2]

Especially, there is some potential of conflict with the `getindex` method when creating `1x1`-matrices:

In [19]:
f = R[2]^2 # the first variable of `R` squared

y^2

In [20]:
F = R[2;]^2 # the 1x1-matrix A = (2) squared

[4]

In [21]:
M[1, 2] # the (1,2)-th entry of the matrix M

y

Matrices can further be processed; for instance, we can take minors:

In [22]:
minors(M, 2) # return a list of 2x2-minors of M

3-element Vector{fmpq_mpoly}:
 x^2 - y*z
 x*y^2 - z^2
 -x*z + y^3

...or submatrices:

In [23]:
subM1 = M[1:end, 1:2] # this takes two `range`s as input

[x   y]
[z   x]

In [24]:
subM2 = M[1:2, [1,3]] # this takes a `range` and a `Vector{Int}` as input

[x     z]
[z   y^2]

If we want a specific minor of a matrix, we have to specify the 
corresponding square submatrix first and then take the determinant:

In [25]:
det(subM2)

x*y^2 - z^2

### Ideals in polynomial rings

We can create ideals in polynomial rings like `R` from a list of generators as follows:

In [14]:
I = ideal(R, [x^2 - y*z, x*y^2 - z^2, -x*z + y^3])

ideal(x^2 - y*z, x*y^2 - z^2, -x*z + y^3)

Any ideal can be asked for the generators with which it was once constructed:

In [5]:
g = gens(I)

3-element Vector{fmpq_mpoly}:
 x^2*z - y^2
 -x^5 + x*z
 x^3 - z^2

We can test ideal membership in `I` for elements of `R`:

In [16]:
x + 1 in I

false

In [18]:
3*(x^2 - y*z) + z*(x*y^2 - z^2) in I

true

Moreover, for any given element $f \in I$, we can ask 
for the coefficients of $f$ in a linear combination of the generators of $I$:

In [19]:
coordinates(3*(x^2 - y*z) + z*(x*y^2 - z^2), I)

[3   z   0]

Note that such "coordinates" are not unique!

## Geometry: Varieties and their singular loci

For the computer, an affine variety is usually modeled by the ideal $I \subset R$ defining it. 
In Oscar, we even have data types for varieties (later!), but we will stick with the algebraic 
side for the moment, thinking of it geometrically. 

Let us come back to one of our previous examples...

In [33]:
R, (x,y) = QQ["x", "y"]
f = x*y*(1-x-y)
I = ideal(R, f)

ideal(-x^2*y - x*y^2 + x*y)

... and let us find out 
what we can about the underlying variety.

In [34]:
dim(I)

1

So the variety defined by `I` is an algebraic curve. Does it have singular points? Let's apply the jacobian criterion!

In [35]:
g = gens(I)
Dg = jacobi_matrix(g)

[-2*x*y - y^2 + y]
[-x^2 - 2*x*y + x]

In [39]:
J = ideal(R, minors(Dg, 1)) + I # Apply the jacobian criterion

ideal(-2*x*y - y^2 + y, -x^2 - 2*x*y + x, -x^2*y - x*y^2 + x*y)

Often, the output is not very readable. Since we are only 
interested in the geometric locus (and not the scheme theoretic 
structure), there is some hope that the radical is more simple.
But be warned! Taking the radical is in general a very 
expensive procedure. 

In [40]:
radical(J)

ideal(y^2 - y, x*y, x^2 - x)

No luck in this case. As a next step, let us try to decompose `J` using the `primary_decomposition`:

In [41]:
primary_decomposition(J)

3-element Vector{Tuple{MPolyIdeal{fmpq_mpoly}, MPolyIdeal{fmpq_mpoly}}}:
 (ideal(y, x), ideal(x, y))
 (ideal(y - 1, x), ideal(y - 1, x))
 (ideal(y, x - 1), ideal(y, x - 1))

We can wrap this in a new function:

In [50]:
function radical_of_singular_locus(I) 
    d = dim(I)                     # the dimension of the ideal 
    R = base_ring(I)               # the ring in which `I` lives
    g = gens(I)
    Dg = jacobi_matrix(g)
    n = ngens(R)                   # the number of variables of `R`
    Ising = I + ideal(R, minors(Dg, n-d))
    return radical(Ising)
end

radical_of_singular_locus (generic function with 1 method)

Let's apply it to our previous ideal and see whether it works.

In [51]:
radical_of_singular_locus(I)

ideal(z - 1, x^2 - 2*x - y^2)

## Local rings in OSCAR

In Oscar, we can localize polynomial rings at geometric points $p \in \mathbf A^n$. This 
is important in singularity theory, since geometrically the algebraic procedure of localization 
corresponds to passing from an affine variety to its germ at a point.

In [43]:
R, (x,y,z) = QQ["x", "y", "z"]
U = complement_of_ideal(R, [0,0,0]) # The multiplicative set of functions not vanishing at the origin
L, map_from_R_to_L = Localization(R, U)

(localization of Multivariate Polynomial Ring in x, y, z over Rational Field at the complement of maximal ideal corresponding to point with coordinates fmpq[0, 0, 0], Map with following data
Domain:
Multivariate Polynomial Ring in x, y, z over Rational Field
Codomain:
localization of Multivariate Polynomial Ring in x, y, z over Rational Field at the complement of maximal ideal corresponding to point with coordinates fmpq[0, 0, 0])

Just like, for example, `Int` is casted to `Float` when evaluating `1 + 1.3`, 
elements of the original ring `R` are automatically casted to 
elements of `L` whenever the context requires it; we can freely 
do arithmetic mixing elements of both rings.

In [44]:
a = x + inv(L(y-1))

(x*y - x + 1)//(y - 1)

In [45]:
parent(a) == L

true

Also, mathematical comparison works as expected:

In [46]:
one(L) == one(R)

true

For comparison of the objects in the computer (i.e. in memory), there is the `===` operator. It gives another result:

In [47]:
one(L) === one(R)

false

Let us come back to geometry. 
We set up the following variety. X is the union of a 
hyperplane H = {z = 1} and the cylinder over a smooth
conic C = {x(x-2)-y^2 = 0}. Both components are smooth, 
but their union is singular along the intersection of 
both components.

In [48]:
h = z-1
f = x*(x-2)-y^2
I = ideal(R, f*h)

ideal(x^2*z - x^2 - 2*x*z + 2*x - y^2*z + y^2)

Let us use our above function to find out about the 
singular locus:

In [58]:
J = radical_of_singular_locus(I)

ideal(z - 1, x^2 - 2*x - y^2)

In [60]:
J_loc = L(J)

ideal in localization of Multivariate Polynomial Ring in x, y, z over Rational Field at the complement of maximal ideal corresponding to point with coordinates fmpq[0, 0, 0] generated by the elements (z - 1)//1, (x^2 - 2*x - y^2)//1

`J` is now an ideal in the ring `L`. 

In [62]:
base_ring(J_loc) == L

true

In particular, it has another data type working in the background!

In [64]:
typeof(J_loc)

MPolyLocalizedIdeal{MPolyLocalizedRing{FlintRationalField, fmpq, FmpqMPolyRing, fmpq_mpoly, MPolyComplementOfKPointIdeal{FlintRationalField, fmpq, FmpqMPolyRing, fmpq_mpoly}}, MPolyLocalizedRingElem{FlintRationalField, fmpq, FmpqMPolyRing, fmpq_mpoly, MPolyComplementOfKPointIdeal{FlintRationalField, fmpq, FmpqMPolyRing, fmpq_mpoly}}}

In [65]:
typeof(J)

MPolyIdeal{fmpq_mpoly}

Nevertheless, `J_loc` should behave just like any ideal should behave, i.e. 
one should be able to ask for a set of generators, one should be able 
to test ideal membership of elements, etc. Let us check this in our 
geometric example.

Locally at the point p = (0,0,0), we only have the 
smooth component C and its singular intersection 
with H is far away. Hence, the localized ideal of 
the singular locus should be trivial. Indeed, 
we find:

In [66]:
one(L) in J_loc

true

We can also try to use our above method on the localization of `I` directly:

In [71]:
I_loc = L(I)

ideal in localization of Multivariate Polynomial Ring in x, y, z over Rational Field at the complement of maximal ideal corresponding to point with coordinates fmpq[0, 0, 0] generated by the elements (x^2*z - x^2 - 2*x*z + 2*x - y^2*z + y^2)//1

In [70]:
radical_of_singular_locus(I_loc) 

ideal in localization of Multivariate Polynomial Ring in x, y, z over Rational Field at the complement of maximal ideal corresponding to point with coordinates fmpq[0, 0, 0] generated by the elements 1//1